## Steps
<li>importing libraries

<li>creating S3 bucket 
<li> Mapping train and test data in S3

In [38]:
import sagemaker
import boto3
from sklearn.model_selection import train_test_split
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session
import numpy as np

In [14]:
bucket_name = 'mukulproject'
my_region = boto3.session.Session().region_name
print(my_region)

us-west-2


# creating s3 bucket

In [15]:
#creating s3 bucket using code but also can be created manually in AWS console
s3 = boto3.resource('s3')
try:
    if my_region == 'us-west-2':
        s3.create_bucket(Bucket=bucket_name,CreateBucketConfiguration={'LocationConstraint': 'us-west-2'})
    print("S3 bucket created successfully")
except Exception as e:
    print("S3 error: ", e )
    


S3 bucket created successfully


In [16]:
#set an output path to save the trained model
prefix = "xgboost-as-a-built-in-algo" #using xgboost builtin  algorithm
output_path = 's3://{}/{}/output'.format(bucket_name,prefix)
output_path

's3://mukulproject/xgboost-as-a-built-in-algo/output'

# Downloading the data and storing in S3

In [17]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv","bank_clean.csv")
    print("Success: downloaded bank_clean.csv")
except Exception as e:
    print("Data Load error: ", e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print("Success: data loaded to dataframe")
except Exception as e:
    print("Data load error: ", e)
    

Success: downloaded bank_clean.csv
Success: data loaded to dataframe


## Train Test split

In [19]:
train_data, test_data = train_test_split(model_data,test_size=0.3)
train_data.shape, test_data.shape

((28831, 61), (12357, 61))

In [25]:
#saving train and test data into s3 bucket
import os 
pd.concat([train_data['y_yes'], train_data.drop(['y_no','y_yes'], axis=1)],axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'train/train.csv')).upload_file('train.csv')
#creating input train path
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [26]:
pd.concat([test_data['y_yes'], test_data.drop(['y_no','y_yes'], axis=1)],axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'test/test.csv')).upload_file('test.csv')
#creating input test path
s3_input_test = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

## Build XGboost Model

In [27]:
#automatically looks for the xgboost image uri  and builds a xgboost container
#algorithm is an inbuilt algorithm present in amazon sagemaker
container = get_image_uri(boto3.Session().region_name, 
                         'xgboost',
                         repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [28]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [30]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role() #IAM role,
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge' #to the training lit bit faster, 
                                          train_volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [32]:
estimator.fit({'train':s3_input_train, 'validation':s3_input_test})

2021-11-29 18:21:25 Starting - Starting the training job...
2021-11-29 18:21:48 Starting - Launching requested ML instancesProfilerReport-1638210084: InProgress
...
2021-11-29 18:22:20 Starting - Insufficient capacity error from EC2 while launching instances, retrying!............
2021-11-29 18:24:21 Starting - Preparing the instances for training............
2021-11-29 18:26:24 Downloading - Downloading input data...
2021-11-29 18:26:56 Training - Training image download completed. Training in progress.
2021-11-29 18:26:56 Uploading - Uploading generated training model.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:r

## Deploying the model

In [34]:
xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-------!

## predictions on the test data


In [39]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_types = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


(12357,)


In [40]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 90.0%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10820)    36% (169)
Purchase        9% (1070)     64% (298) 



In [41]:
#deleting the endpoints
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


[{'ResponseMetadata': {'RequestId': 'MWSES3B7K3QGG8NP',
   'HostId': 'XHQ/lu6PVd14v8Hfv5bqB39Aua95P6x4ZeNzU/Qz/sAyLXXUkll/Hc6SehiV9bc5H6XEgYq5lTg=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'XHQ/lu6PVd14v8Hfv5bqB39Aua95P6x4ZeNzU/Qz/sAyLXXUkll/Hc6SehiV9bc5H6XEgYq5lTg=',
    'x-amz-request-id': 'MWSES3B7K3QGG8NP',
    'date': 'Mon, 29 Nov 2021 18:44:34 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2021-11-29-18-21-24-764/profiler-output/system/training_job_end.ts'},
   {'Key': 'xgboost-as-a-built-in-algo/train/train.csv'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2021-11-29-18-21-24-764/output/model.tar.gz'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2021-11-29-18-21-24-764/profiler-output/framework/training_job_end.ts'},
   {'Key': 'xgboost